# Importação de pacotes

In [2]:
import pandas as pd
from pycaret.regression import *

# Carregando os dados

In [8]:
# carregando os dados para o formato DataFrame
df = pd.read_csv("insurance_clean.csv", sep=";")

# imprimindo as 5 primeiras linhas
df.head()

,age,sex,bmi,children,smoker,region,charges,age_discretized,bmi_description,children_description,charges_log,smoker_bmi
0,19,female,27.900,0,yes,southwest,16884.92400,0.0,overweight,no children,9.734176,yes_not_obese
1,18,male,33.770,1,no,southeast,1725.55230,0.0,obese,up to 3 children,7.453302,no
2,28,male,33.000,3,no,southeast,4449.46200,1.0,obese,up to 3 children,8.400538,no
3,33,male,22.705,0,no,northwest,21984.47061,1.0,normal,no children,9.998092,no
4,32,male,28.880,0,no,northwest,3866.85520,1.0,overweight,no children,8.260197,no


# Separando dados em treino e teste

A função `sample()` do Pandas DataFrame é usada para obter uma amostra aleatória de linhas ou colunas de um DataFrame. A função retorna um novo DataFrame que contém uma fração aleatória de linhas ou colunas do DataFrame original.

O parâmetro `frac` determina a fração de linhas que você deseja retornar em vez de um número fixo. Neste caso é usado somente 10% do dados para formar o conjunto de teste e 90% para formar os nossos dados de treino.

In [9]:
# separando os dados em teste e treino
test = df.sample(frac=0.10)
train = df.drop(test.index)

# redefinindo o index do DataFrame descartando o indice anterior (drop=True) e realizando a alteração no próprio DataFrame (inplace=True) 
test.reset_index(inplace=True, drop=True)
train.reset_index(inplace=True, drop=True)

In [10]:
# verificando o formato dos conjuntos
print(f"Shape do conjunto de treino {train.shape}")
print(f"Shape do conjunto de teste {test.shape}")

Shape do conjunto de treino (1204, 12)
Shape do conjunto de teste (134, 12)


# Setup

O método `setup()` do PyCaret é usado para preparar os dados para modelagem de aprendizado de máquina. Ele executa várias etapas de pré-processamento de dados, como:

* Separar o conjunto de dados em treinamento e teste.
* Lidar com valores ausentes e valores discrepantes (outliers).
* Codificar variáveis categóricas.
* Normalizar ou padronizar variáveis numéricas.
* Selecionar recursos (features) importantes.
* Balancear os dados.

O método `setup()` retorna um objeto contendo várias informações sobre o pré-processamento de dados que foi realizado, bem como uma pré-visualização dos dados transformados. A partir desse objeto, você pode criar, treinar e avaliar modelos de aprendizado de máquina usando o PyCaret.

In [17]:
# criando o setup do PyCaret
reg = setup(data=train,
            target="charges",
            train_size=0.7,
            preprocess=True,
            normalize=True,
            fold_strategy="kfold",
            fold=10,
            session_id=123)

,Description,Value
0,session_id,123
1,Target,charges
2,Original Data,"(1204, 12)"
3,Missing Values,False
4,Numeric Features,4
5,Categorical Features,7
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(842, 24)"


# Compare models

O método `compare_models` do PyCaret é usado para comparar o desempenho de vários modelos de aprendizado de máquina em um conjunto de dados, usando uma métrica de avaliação específica.

Ao chamar o método `compare_models`, o PyCaret executa automaticamente um experimento de modelagem de aprendizado de máquina usando vários algoritmos de modelagem diferentes. Ele usa um método de validação cruzada para treinar e avaliar cada modelo, dividindo o conjunto de dados em vários subconjuntos e avaliando o desempenho do modelo em cada subconjunto.

O resultado do `compare_models` é uma tabela de classificação de desempenho dos modelos, ordenados pela métrica de avaliação escolhida.

In [19]:
best = compare_models(round=2, sort="MSE")

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,78.69,44156.03,167.71,1.00,0.01,0.01,0.02
dt,Decision Tree Regressor,88.95,108387.77,277.79,1.00,0.01,0.01,0.00
rf,Random Forest Regressor,72.78,144401.04,285.76,1.00,0.01,0.00,0.06
et,Extra Trees Regressor,104.44,287865.61,432.30,1.00,0.02,0.01,0.06
lightgbm,Light Gradient Boosting Machine,166.40,746623.83,659.45,0.99,0.03,0.01,0.01
ada,AdaBoost Regressor,631.14,794389.49,874.36,0.99,0.23,0.15,0.02
llar,Lasso Least Angle Regression,2201.86,9475303.50,3073.85,0.93,0.55,0.37,0.00
ridge,Ridge Regression,2208.82,9504097.45,3078.08,0.93,0.56,0.37,0.00
lasso,Lasso Regression,2203.22,9505403.95,3078.36,0.93,0.56,0.37,0.00
br,Bayesian Ridge,2207.30,9506174.58,3078.43,0.93,0.56,0.37,0.00


# Métricas de regressão